In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import matplotlib.pyplot as plt # For plotting training history

In [2]:
# --- Step 1: Define Paths and Parameters ---

# Define the root directory where 'train' and 'test' folders are located
# IMPORTANT: Adjust 'fer2013' if your main dataset folder has a different name
base_data_dir = 'fer2013'

train_data_dir = os.path.join(base_data_dir, 'train')
test_data_dir = os.path.join(base_data_dir, 'test')

# VGG16 was originally trained on ImageNet images which are 224x224 pixels.
# We need to resize our images to this size.
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3 # VGG16 expects 3 color channels (RGB)

# Batch size: How many images to process at once during training.
# Adjust based on your computer's RAM/GPU memory. 32 is a common starting point.
BATCH_SIZE = 32

# Number of classes (emotions) you are classifying
NUM_CLASSES = 7 # angry, disgust, fear, happy, neutral, sad, surprise

print(f"Train data directory: {train_data_dir}")
print(f"Test data directory: {test_data_dir}")
print(f"Target image size for VGG16: {IMG_HEIGHT}x{IMG_WIDTH}")
print(f"Number of emotion classes: {NUM_CLASSES}")

Train data directory: fer2013\train
Test data directory: fer2013\test
Target image size for VGG16: 224x224
Number of emotion classes: 7


In [3]:
# --- Step 2: Prepare Your Data with ImageDataGenerator ---

# 2.1. Create a Data Generator for Training Data
# We'll apply data augmentation to the training set to make our model more robust.
# VGG16 requires a specific preprocessing function.
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # <-- This is CRITICAL for VGG16!
    rotation_range=20,     # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2, # Randomly shift images horizontally
    height_shift_range=0.2,# Randomly shift images vertically
    shear_range=0.2,       # Apply shear transformations
    zoom_range=0.2,        # Randomly zoom in/out
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'    # Strategy for filling in new pixels created by transformations
)

# 2.2. Create a Data Generator for Test (and Validation) Data
# For test/validation data, we only need to resize and apply VGG16 preprocessing.
# No augmentation should be applied to test/validation data.
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input # <-- Still CRITICAL for VGG16!
)

# 2.3. Flow Images from Directories using the Generators
print("\nLoading training images...")
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH), # Resize images to VGG16 input size
    batch_size=BATCH_SIZE,               # Number of images per batch
    class_mode='categorical',            # 'categorical' for one-hot encoded labels (e.g., [0,0,1,0,0,0,0] for 'fear')
    shuffle=True                         # Shuffle training data for better training
)

print("\nLoading test images...")
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH), # Resize images
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False                        # Do NOT shuffle test data for consistent evaluation
)

# You can get the class names that the generator inferred from your folder names
class_names = list(train_generator.class_indices.keys())
print(f"\nClasses detected by generator: {class_names}")
print(f"Number of training images: {train_generator.samples}")
print(f"Number of test images: {test_generator.samples}")


Loading training images...
Found 28709 images belonging to 7 classes.

Loading test images...
Found 7178 images belonging to 7 classes.

Classes detected by generator: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Number of training images: 28709
Number of test images: 7178


In [4]:
# --- Step 3: Load VGG16 and Build Your Model ---

# 3.1. Load the Pre-trained VGG16 Model (Convolutional Base)
# - weights='imagenet': Use weights pre-trained on the ImageNet dataset.
# - include_top=False: This is crucial! It removes the original fully-connected layers (the "head")
#                      that were used for ImageNet's 1000 classes.
# - input_shape: Specifies the input shape for your images (height, width, channels).
base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

print("\nVGG16 base model loaded successfully.")
base_model.summary() # Review the layers of the VGG16 base

# 3.2. Freeze the Layers of the Base Model
# Freezing means that the weights of these layers will NOT be updated during training.
# We are using VGG16 as a fixed feature extractor.
for layer in base_model.layers:
    layer.trainable = False
print("\nAll layers in the VGG16 base model have been frozen.")

# 3.3. Add Your Custom Classification Head
# We will add new layers on top of the frozen VGG16 base.
# These are the only layers whose weights will be trained initially.

x = base_model.output # Get the output tensor from the VGG16 base
x = Flatten()(x)      # Flatten the 3D output of the convolutional layers into a 1D vector

# Add a Dense (fully connected) layer for feature processing
x = Dense(256, activation='relu')(x) # 256 neurons, using ReLU activation

# Add a Dropout layer for regularization (helps prevent overfitting)
# 0.5 means 50% of neurons will be randomly "dropped" during each training step
x = Dropout(0.5)(x)

# Add the final output layer for classification
# NUM_CLASSES neurons (one for each emotion), 'softmax' activation for probability distribution
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# 3.4. Create the Final Model
# This combines the VGG16 base (input) with your new classification head (output).
model = Model(inputs=base_model.input, outputs=predictions)

# 3.5. Compile the Model
# - optimizer: How the model updates its weights (Adam is a good general choice)
#   - learning_rate: A small learning rate is important when using pre-trained models.
# - loss: The function the model tries to minimize. 'categorical_crossentropy' is used
#         when your labels are one-hot encoded (which `flow_from_directory` with `categorical_mode` does).
# - metrics: What to monitor during training (e.g., accuracy).
model.compile(optimizer=Adam(learning_rate=0.0001), # Use a relatively small learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("\n--- Final Model Summary (VGG16 base + custom head) ---")
model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step

VGG16 base model loaded successfully.


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

...

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 21,139,271 (80.64 MB)

 Trainable params: 6,424,583 (24.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [5]:
# --- Step 4: Train Your Model ---

# 4.1. Define Callbacks (Optional but Recommended)
# Callbacks are tools to help manage your training process.
# - EarlyStopping: Stops training if the validation loss doesn't improve for a few epochs.
#                  This prevents overfitting and saves time.
# - ModelCheckpoint: Saves the best performing model (based on a metric like validation accuracy).
early_stopping = EarlyStopping(monitor='val_loss', # Monitor validation loss
                               patience=5,         # Stop if val_loss doesn't improve for 5 epochs
                               restore_best_weights=True) # Load the best weights found

model_checkpoint = ModelCheckpoint('best_vgg16_emotion_classifier.keras', # Name of the file to save
                                   save_best_only=True, # Only save if current model is better
                                   monitor='val_accuracy', # Monitor validation accuracy
                                   mode='max',          # We want to maximize validation accuracy
                                   verbose=1)           # Show messages when saving

# 4.2. Set Number of Epochs
# An epoch is one complete pass through the entire training dataset.
# You might need to experiment with this.
EPOCHS = 20 # Start with a reasonable number, EarlyStopping will prevent overtraining

print(f"\nStarting model training for {EPOCHS} epochs...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator, # Use the test_generator for validation
    callbacks=[early_stopping, model_checkpoint] # Pass our defined callbacks
)

print("\nTraining complete!")


Starting model training for 20 epochs...


C:\Users\user\anaconda3\envs\ai_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
  5/898 ━━━━━━━━━━━━━━━━━━━━ 59:29 4s/step - accuracy: 0.1666 - loss: 11.6236

KeyboardInterrupt: 